#**Chain of Thoughts Prompting: BLOOM**

###Загрузка модели и установка необходимых пакетов

Сначала надо загрузить в рабочую директорию файл test.jsonl из датасета GMS8K и файл utilits.py из материалов статьи (Wang et al. 2023) (они же есть в репозитории на гитхабе), затем установить нужные пакеты и загрузить модель.

In [1]:
%pip install -q petals

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [19]:
import torch
import transformers
from transformers import BloomTokenizerFast
from petals import DistributedBloomForCausalLM

import dataclasses
import json
import jsonlines
import time
import utils #файл utilits.py из материалов статьи (Wang et al. 2023)
import re

К сожалению, предоставленных Google Collab GPU недостаточно для использования модели BLOOM-176B, которую нужно использовать по условию. 

BLOOM-176B генерировала одно продолжение до 100 новых токенов в течение 4-6 минут. Так как для ансамблированной СоТ нужно несколько генераций, а для уверенных результатов нужно сравнить ответы greedy prompting и Chain of Thoughts prompting для всей тестовой части GSM датасета, такой скорости генерации при бесплатных 12 часов GPU в Google Collab хватило бы примерно только на генерацию по 5 ответов на 50 задач, что означало бы с большой вероятностью случайные результаты исследования.

Поэтому **размер модели, используемой в этой тетрадке: 7.1B параметров**, но код и весь алгоритм действий применимы, естественно, и для BLOOM-176B (разница будет заключаться в строке с MODEL_NAME)

Прошу прощения за несоответствие условию размера модели, мне не удалось решить проблему скорости генерации с большой BLOOM.


In [ ]:
#MODEL_NAME = "bigscience/bloom-petals" #для BLOOM-176B

MODEL_NAME = "bigscience/bloom-7b1-petals" #для BLOOM-71B
tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME)
model = DistributedBloomForCausalLM.from_pretrained(MODEL_NAME)
model = model.cuda()

In [ ]:
inputs = tokenizer('A cat in French is "', return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=100).cuda()
print(tokenizer.decode(outputs[0]))

In [7]:
tokenizer.decode(outputs[0])

'A cat in French is "chat", and in English is "cat". The word "cat" is a noun, and "chat" is a verb. The word "cat" is a noun, and "chat" is a verb. The word "cat" is a noun, and "chat" is a verb. The word "cat" is a noun, and "chat"'

###Создание промптов и генерация ответов на задачи

Для того, чтобы полученные результаты можно было сравнить с результатами работы (Wang et al., 2023), используем для начала промптинг из данной работы -- 8 математических задач с ответами *(Wang et al., 2023: 35)*. Эти 8 задач являются промтингом для каждой из задач, ответ на которую будет генерировать BLOOM.

В качестве целевых задач возьмём, как в (Wei et al., 2022) и в Wang et al., 2023), тестовую часть датасета GSM8K. Код ниже создаёт списки **input_list** (промпт+задача1, промпт+задача2, и т. д.), **tasks** (задача1, задача2, ...), **answer_list** (верные ответы на задачи).


In [8]:
INPUT_FILE = 'test.jsonl'

@dataclasses.dataclass
class Example:
  question: str
  answer: str
  thought: str


GSM_EXAMPLES = [
    Example(
        question='There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?',
        answer='6',
        thought='There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.',
    ),
    Example(
        question='If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?',
        answer='5',
        thought='There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.',
    ),
    Example(
        question='Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?',
        answer='39',
        thought='Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39.',
    ),
    Example(
        question='Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?',
        answer='8',
        thought='Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8.',
    ),
    Example(
        question='Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?',
        answer='9',
        thought='Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9.',
    ),
    Example(
        question='There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?',
        answer='29',
        thought='There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29.',
    ),
    Example(
        question='Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?',
        answer='33',
        thought='Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls.',
    ),
    Example(
        question='Olivia has $23. She bought five bagels for $3 each. How much money does she have left?',
        answer='8',
        thought='Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8.',
    ),
]

tasks = []
input_list = []
answer_list = []

with jsonlines.open(INPUT_FILE) as file:
    for line in file:
      question = line['question']
      answer = line['answer']

      prompt = ''
      for j, ex in enumerate(GSM_EXAMPLES):
        prompt += 'Q: ' + ex.question + '\nA: ' + ex.thought + ' The answer is ' + ex.answer  + '.\n\n'
      input_list.append(prompt + 'Q: ' + question.replace('\\n', '\n') + '\nA:')
      tasks.append(question)
      answer_list.append(re.findall(re.compile("\#\#\#\#(.*)"), answer)[0].strip())

print(len(input_list))
print(input_list[0])

1319
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denn

In [9]:
tasks[12]

'Carlos is planting a lemon tree. The tree will cost $90 to plant. Each year it will grow 7 lemons, which he can sell for $1.5 each. It costs $3 a year to water and feed the tree. How many years will it take before he starts earning money on the lemon tree?'

In [10]:
answer_list[12]

'13'

BLOOM не позволяет "доставать" несколько вариантов сгенерированных продолжений строки (в отличие, например, от GPT-3, что сделано в (Wang et al., 2023)), поэтому ансамблированной CoT в данном случае будем считать самый популярный ответ среди нескольких (независимых) генераций модели ("попыток"  решить задачу). 

В силу ограниченности GPU будем генерировать по 5 вариантов решения задачи, хотя для большей уверенности в результатах лучше установить RETRY_SIZE побольше (и провести эксперименты по его влиянию на значения финальной accuracy).

Результаты будем записывать в файл **'bloom_gsm8k_output_sc.jsonl'**, каждая строка которого имеет вид 
`{"input":"общий промпт из 8 задач с ответами+таргетная задача", "output":["решение и ответ1", "решение и ответ2", ..., "решение и ответ5"], "answer":"правильный ответ"}`

In [ ]:
OUTPUT_PATH = 'bloom_gsm8k_output_sc.jsonl'

start_time = time.time()
RETRY_SIZE = 5

with open(OUTPUT_PATH, 'w') as outfile:
  for i in range(len(input_list)):
    outs = []
    for j in range(RETRY_SIZE):
      prompt = input_list[i]
      inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].cuda()
      outputs = model.generate(inputs, max_new_tokens=100)
      reply = tokenizer.decode(outputs[0])
      reply = reply.replace(prompt+' ', '')
      reply = reply[:reply.find('\nA:')]
      print(reply)

      #self-consistency: записываем все ответы
      outs.append(reply)
      
    pred = {}
    pred['input'] = tasks[i]
    pred['output'] = outs
    pred['answer'] = answer_list[i]
    json.dump(pred, outfile)
    print('i m writing')
    outfile.write('\n')
    print('time: ', time.time() - start_time)

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. So she has 16 eggs - 3 = 13 eggs. She sells the remainder at the farmers' market for $2 per fresh duck egg. So she has 13 eggs - 2 = 11 eggs. The answer is 11.

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. So she has 16 * 3 = 48 eggs. She sells the remainder at the farmers' market for $2 per fresh duck egg. So she has 48 - 2 = 46 eggs. The answer is 46.

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. So she has 16 - 3 = 13 eggs. She sells the remainder at the farmers' market for $2 per fresh duck egg. So she makes 13 x 2 = 26 dollars. The answer is 26.

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. 

**Итого:** инпутами модели стали 102 задачи из тестовой части GSM, генерация ответов на них длилась более 6 часов.

**Предварительные наблюдения (без вычислений):** 
1. Лимит в 100 максимальных токенов оказался неудачным для части задач, иногда (в меньшинстве случаев) модель не может уложить всю цепочку рассуждений и ответ в 100 токенов. В этом случае ответ пустой ("").
2. Иногда модель просто "не доходит" до ответа, даже если место для новых токенов ещё осталось. Скорее всего это связано с слишком маленьким размером модели. В этом случае ответ пустой ("").
3. На некоторые арифметически простые задачи (требующие небольшого количества шагов решения) все 5 сгененрированных моделью решений и ответов являются (почти) идентичными, но таких задач не очень много.

In [30]:
len('Jerome started with 4 friends. The first friend pressed on the doorbell 20 times. The second friend pressed on the doorbell 1/4 times more than the first friend. The third friend pressed on the doorbell 10 times more than the fourth friend. The fourth friend pressed on the doorbell 60 times. So the first friend pressed on the doorbell 20 + 1/4 + 10 = 40 times. The second friend pressed on the doorbell 20 + 1/4 = 40 times. Th'.split())

81

###Результаты

**1. Результаты для greedy Chain of Thoughts prompting**

Для оценки результатов greedy prompting возьмем из файла с результатами self-consistency prompting для каждого задания первый непустой ответ. (Можно было бы также взять случайный непустой ответ.)

Пустые ответы не будем учитывать, потому что часть из них вызывана ограничением на количество новых токенов. Мне кажется неправильно снижать accuracy модель за некраткость рассуждений при таких условиях эксперименета.

Будем считать долю верных ответов не только от общего количества задач, которые модель решала, но и от общего количества задач, которые модель хотя бы в одну из попыток решила -- то есть дошла до непустого ответа (The answer is ....).

In [26]:
right_total = 0

with open('bloom_gsm8k_output_sc.jsonl', 'r') as f:
  lines = f.readlines()

correct = 0
for line in lines:
  data = json.loads(line)
  pred_list = []
  answers = []
  for out in data['output']:
    answers.append(utils.get_ans(out))
  ans = ''
  for answer in answers:
    if answer != "":
      right_total += 1
      ans = answer
      break
  target = data['answer']
  if utils._is_float(target) and utils._is_float(ans):
    if abs(float(target) - float(ans)) <= 1e-5:
      correct += 1
  elif str(target) == str(ans):
    correct += 1

total = len(lines)
print("Результаты для  ансамблированного Chain of Thoughts Prompting (self-concistency CoT)\n")
print("Количество верных ответов: ", correct, "\nВсего заданий: ", total, "\nДоля верных ответов: ", correct/total, "\n")
print("Количество верных ответов: ", correct, "\nВсего заданий, на которые модель выдала хотя бы один непустой ответ (дошла до ответа): ", right_total, "\nДоля верных ответов в этом случае: ", correct/right_total)

Результаты для  ансамблированного Chain of Thoughts Prompting (self-concistency CoT)

Количество верных ответов:  5 
Всего заданий:  102 
Доля верных ответов:  0.049019607843137254 

Количество верных ответов:  5 
Всего заданий, на которые модель выдала хотя бы один непустой ответ (дошла до ответа):  88 
Доля верных ответов в этом случае:  0.056818181818181816


**2. Результаты для ансамблированного Chain of Thoughts Prompting (self-concistency CoT)**

In [32]:
right_total = 0

with open('bloom_gsm8k_output_sc.jsonl', 'r') as f:
  lines = f.readlines()

correct = 0
for line in lines:
  data = json.loads(line)
  pred_list = []
  for pred in data['output']:
    ans = utils.get_ans(pred)
    if ans:
      pred_list.append(ans)
  if not pred_list:
    continue
  maj_ans = utils.get_maj(pred_list)
  right_total += 1
  target = data['answer']
  if utils._is_float(target) and utils._is_float(maj_ans):
    if abs(float(target) - float(maj_ans)) <= 1e-5:
      correct += 1
  elif str(target) == str(maj_ans):
    correct += 1

total = len(lines)
print("Результаты для greedy Chain of Thoughts prompting\n")
print("Количество верных ответов: ", correct, "\nВсего заданий: ", total, "\nДоля верных ответов: ", correct/total, "\n")
print("Количество верных ответов: ", correct, "\nВсего заданий, на которые модель выдала хотя бы один непустой ответ (дошла до ответа): ", right_total, "\nДоля верных ответов в этом случае: ", correct/right_total)

Результаты для greedy Chain of Thoughts prompting

Количество верных ответов:  4 
Всего заданий:  102 
Доля верных ответов:  0.0392156862745098 

Количество верных ответов:  4 
Всего заданий, на которые модель выдала хотя бы один непустой ответ (дошла до ответа):  88 
Доля верных ответов в этом случае:  0.045454545454545456


Доля верных ответов greedy CoT prompting: **0.045**

Доля верных ответов self-consistency prompting (ансамблированный CoT): **0.057**

**Вывод о том, какой из вариантов Chain-of-Thoughts промптинга показывает лучшие результаты, сделать нельзя.** 

Полученные значения метрик являются **случайными**, потому что доля верных ответов очень низка, отличие в 1 верный ответ при 88 задачах -- случайность. Вероятно, главная причина -- в маленьком размере модели и в недостатке GPU для обучения.

В 24 задаче модель BLOOM не дошла до ответа ни в одной из пяти попыток, чаще всего потому, что достигла лимита в 100 новых токенов. Можно было бы, например, с помощью регулярных выражений искать последнее число в сгенерированннном отрывке, но по моим наблюдениям для данного эксперимента это никак не повлияло бы на результаты (все числа из нерешенных до конца задач не совпадали с ответом).

Что получилось: 
* код, аналогичный используемому в предыдущих работать по СоТ, но для BLOOM (подойдёт для для любого размера);
* применение идеи ансамбля к BLOOM;
* осознание важности выбора параметров RETRY_SIZE, max_new_tokens и других параметров (см. План)

Что не получилось:
* из-за ограничености GPU довести первичный (базовый) эксперимент до конца: взять большую модель, разрешить ей генерировать много токенов, выдать ответы на все задачи тестового GSM
* провести эксперименты, проверяющие вляние отдельных параметров на результат
* имеющее смысл сравнение с результатами других моделей (при получившихся результатах можно провести только не имеющее смысл сравнение, BLOOM тогда окажется худшей из всех моделью)


**Как можно было бы улучшить результаты ансамблированного СоТ:**

  * для каждого ответа посмотреть на его Chain of Thoughts и посчитать среди них уникальные. Выбрать как фильный ответ, у которого больше всего уникальных СоТ (можно считать не абслютную идентичность, а, например, использовать расстояние Левенштейна). В чём идея: модель может много раз нагенерировать неверный ответ при одной и той же неверной цепочки рассуждений. Если же цепочки размышлений разные, а ответ одинаковый, есть вероятность, что он верный (это согласуется с базовой идеей self-consistency: есть много путей прийти к одному ответу)

  * попробовать брать ответ из ансамбля ансаблей (каждый ансамбль = новые икс генераций для каждой задачи). Возможно, сработает по логике решающих деревьев -- ансамбли над простыми моделями сильно улучшат качество (и это выгоднее, чем одна сложная модель)

###План следующих экспериментов:

0. Провести **полноценный первый эксперимент (без ограничений GPU)**: BLOOM-176B, весь тестовый датасет, увеличенный лимит на максимальное количество токенов, большие ансамбли (для greedy: попробовать случайный выбор)

1. **Эксперимент по влиянию сета задач для промтинга**: попробовать взять не 8 задач из предыдущих работ, а прозвольные 8/по икс задач каждого из рангов (по количеству шагов в CoT), икс задач, на которые без промптинга модель очень часто даёт неверный ответ, и т. д. (В (Wang et al., 2023) нет сильной зависимости результатов от разных инпут сетов.) Также проверить, влияет ли порядок задач промптинга, в котором их видит модель, и количество промптов. 

2. Если нас интересует CoT в целом, обязательно провести **эксперименты на датасетах других естественных языков** (желательно не германских, а типологически разных). Идея CoT и асамблированного СоТ должна по своей логике работать на всех языках. Особенно интересно было бы посмотреть на языки с отличной от 10 базой системы счисления (из крупных языков -- 20 в французском), возможно, результаты будут отличаться.

3. Эксперимент без и **с использованием калькулятора**. В описании датасета GSM указано, что часто модели ошибаются именно в арифметике -- можно подключить калькулятор.

4. Аналогично работе (Wang et al., 2023) посмотреть, что будет:
а) с top-p/top-k sampling или beam search CoT prompting вместо greedy,
б) при разном количестве Sampled Reasoning Paths

###Источники:

* (Wang et al., 2023) -- Xuezhi Wang and Jason Wei and Dale Schuurmans and Quoc V Le and Ed H. Chi and Sharan Narang and Aakanksha Chowdhery and Denny Zhou. Self-Consistency Improves Chain of Thought Reasoning in Language Models // *International Conference on Learning Representations*. 2023.

    *основа кода для оценки моделей, основа кода для генерации промптов, код utilits.py*

* (Wei et al. 2022) -- Jason Wei, Xuezhi Wang, Dale Schuurmans, Maarten Bosma, Brian Ichter, Fei Xia, Ed Chi, Quoc Le, Denny Zhou. Chain-of-Thought Prompting Elicits Reasoning in Large Language Models. 2022.

* https://github.com/bigscience-workshop/petals

  *модель*
* https://github.com/openai/grade-school-math

  *датасет*

P. S. Большое спасибо за интересное задание!